In [2]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=5): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

def wbb_calendar(season):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?dates={}".format(season)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
    datenum = list(map(lambda x: x[:10].replace("-",""),txt))
    date = list(map(lambda x: x[:10],txt))
    
    year = list(map(lambda x: x[:4],txt))
    month = list(map(lambda x: x[5:7],txt))
    day = list(map(lambda x: x[8:10],txt))
    
    data = {"season": season,
            "datetime" : txt,
            "date" : date,
            "year": year,
            "month": month,
            "day": day,
            "dateURL": datenum
            
           }
    df = pd.DataFrame(data)
    df['url']="http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?dates="
    df['url']= df['url'] + df['dateURL']
    return df

wbb_calendar(2018)

,season,datetime,date,year,month,day,dateURL,url
0,2018,2017-11-10T08:00Z,2017-11-10,2017,11,10,20171110,http://site.api.espn.com/apis/site/v2/sports/b...
1,2018,2017-11-11T08:00Z,2017-11-11,2017,11,11,20171111,http://site.api.espn.com/apis/site/v2/sports/b...
2,2018,2017-11-12T08:00Z,2017-11-12,2017,11,12,20171112,http://site.api.espn.com/apis/site/v2/sports/b...
3,2018,2017-11-13T08:00Z,2017-11-13,2017,11,13,20171113,http://site.api.espn.com/apis/site/v2/sports/b...
4,2018,2017-11-14T08:00Z,2017-11-14,2017,11,14,20171114,http://site.api.espn.com/apis/site/v2/sports/b...
...,...,...,...,...,...,...,...,...
131,2018,2018-03-28T07:00Z,2018-03-28,2018,03,28,20180328,http://site.api.espn.com/apis/site/v2/sports/b...
132,2018,2018-03-29T07:00Z,2018-03-29,2018,03,29,20180329,http://site.api.espn.com/apis/site/v2/sports/b...
133,2018,2018-03-30T07:00Z,2018-03-30,2018,03,30,20180330,http://site.api.espn.com/apis/site/v2/sports/b...
134,2018,2018-03-31T07:00Z,2018-03-31,2018,03,31,20180331,http://site.api.espn.com/apis/site/v2/sports/b...


In [3]:
pd.set_option('display.max_colwidth',None)
schedule = {}
for x in range(2001,2022):
    full_schedule = cbb_calendar(x)
    full_schedule.to_json(orient='records')
    schedule[x]=full_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('wbb_calendar.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)    

In [22]:
import json
import urllib
import pandas as pd
import numpy as np
import http
from urllib.error import URLError, HTTPError, ContentTooShortError
from datetime import datetime
import time

def download(url, num_retries=10): 
#     print('Downloading:', url)
    try: 
        time.sleep(1)
        html = urllib.request.urlopen(url).read()
    except (URLError, HTTPError, ContentTooShortError, http.client.HTTPException) as e: 
        print('Download error:', e.reason,url)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                time.sleep(10)
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1)
    return html

def wbb_schedule(year):
    url = "http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?dates={}".format(year)
    resp = download(url=url)
    txt = json.loads(resp)['leagues'][0]['calendar']
#     print(len(txt))
    txt = list(map(lambda x: x[:10].replace("-",""),txt))
    
    ev = pd.DataFrame()
    i=0
    for date in txt:
        print(f"Working on {year}: {i+1} of {len(txt)}, date: {txt[i]}")
        url = "http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates={}&limit=1000".format(date)
        resp = download(url=url)
        if resp is not None:
            events_txt = json.loads(resp)

            events = events_txt['events']
            for event in events:
                bad_keys = ['linescores', 'statistics', 'leaders',  'records']
                for k in bad_keys:
                    if k in event['competitions'][0]['competitors'][0].keys():
                        del event['competitions'][0]['competitors'][0][k]
                    if k in event['competitions'][0]['competitors'][1].keys():
                        del event['competitions'][0]['competitors'][1][k]
                if 'links' in event['competitions'][0]['competitors'][0]['team'].keys():
                    del event['competitions'][0]['competitors'][0]['team']['links']
                if 'links' in event['competitions'][0]['competitors'][1]['team'].keys():
                    del event['competitions'][0]['competitors'][1]['team']['links']    
                if event['competitions'][0]['competitors'][0]['homeAway']=='home':
                    event['competitions'][0]['home'] = event['competitions'][0]['competitors'][0]['team']    
                else: 
                    event['competitions'][0]['away'] = event['competitions'][0]['competitors'][0]['team']
                if event['competitions'][0]['competitors'][1]['homeAway']=='away':
                    event['competitions'][0]['away'] = event['competitions'][0]['competitors'][1]['team']
                else: 
                    event['competitions'][0]['home'] = event['competitions'][0]['competitors'][1]['team']

                del_keys = ['competitors', 'broadcasts','geoBroadcasts', 'headlines']
                for k in del_keys:
                    if k in event['competitions'][0].keys():
                        del event['competitions'][0][k]

                ev = ev.append(pd.json_normalize(event['competitions'][0]))
            i+=1
            ev['season']=year
        else:
            i+=1
            continue
    return ev

schedule_table = pd.DataFrame()
schedule = {}
for x in reversed(range(2002,2003)):
    year_schedule = wbb_schedule(x)
    year_schedule.to_csv(f"wbb/schedules/wbb_games_info_{x}.csv")
    schedule_table = schedule_table.append(year_schedule)
    year_schedule.to_json(orient='records')
    schedule[x]=year_schedule
    
data_dict = {
    key: schedule[key].to_dict(orient='records')
    for key in schedule.keys()
}

with open('wbb_schedule.json','w') as fp:
    json.dump(
    data_dict,
    fp,
    indent=4,
    sort_keys=True)

schedule_table.to_csv('wbb_games_info_2002_2021.csv')
    
# event['competitions'][0]

Working on 2002: 1 of 127, date: 20011109
Working on 2002: 2 of 127, date: 20011111
Working on 2002: 3 of 127, date: 20011115
Working on 2002: 4 of 127, date: 20011116
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011116&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011116&limit=1000
Download error: Bad Gateway http://site.api.esp

Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011203&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011203&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011203&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011203&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011203&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011203&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/

Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011222&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011222&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011222&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011222&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20011222&limit=1000
Working on 2002: 41 of 127, date: 20011223
Working on 2002: 42 of 127, date: 20011227
Working on 2002: 43 of 127, date: 20011228
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/wo

Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20020114&limit=1000
Download error: Bad Gateway http://site.api.espn.com/apis/site/v2/sports/basketball/womens-college-basketball/scoreboard?groups=50&dates=20020114&limit=1000
Working on 2002: 61 of 127, date: 20020115
Working on 2002: 62 of 127, date: 20020116
Working on 2002: 63 of 127, date: 20020117
Working on 2002: 64 of 127, date: 20020118
Working on 2002: 65 of 127, date: 20020119
Working on 2002: 66 of 127, date: 20020120
Working on 2002: 67 of 127, date: 20020121
Working on 2002: 68 of 127, date: 20020122
Working on 2002: 69 of 127, date: 20020123
Working on 2002: 70 of 127, date: 20020124
Working on 2002: 71 of 127, date: 20020125
Working on 2002: 72 of 127, date: 20020126
Working on 2002: 73 of 127, date: 20020127
Working on 2002: 74 of 127, date: 20020128
Working on 2002: 75 of 127, date: 20020129
Working on 2002: 76 of 127, date: 2002013

In [23]:
import glob
import pandas as pd
directoryPath = 'wbb/schedules/'
glued_data = pd.DataFrame()
for file_name in glob.glob(directoryPath+'*.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    glued_data = pd.concat([glued_data,x],axis=0)
    
glued_data.to_csv('wbb_games_info_2002_2021.csv')